## Dynamic Plans

This notebook implements a uav logic instead of a plan and decisions are done online following that logic

In [ ]:
import os
import subprocess
from pymavlink import mavutil
import numpy as np
from collections import OrderedDict

# Custom modules
from vehicle_logic import VehicleLogic
from helpers.change_coordinates import heading_to_yaw,find_spawns,global2local
from helpers.visualization.gazebo_world import update_world
from helpers.math import manhattan_distance
from helpers.navegation_logic import find_path,find_best_waypoint
from helpers.visualization.plots import plot_3d_interactive


Kill all related process

In [2]:
for process in ["QGroundControl", "sim_vehicle.py", "arducopter", "gazebo", "mavproxy"]:
    os.system(f"pkill -9 -f {process}")

Paths

In [3]:
QGC_path='~/QGroundControl.AppImage'
gazebo_word_path="gazebo_worlds/small_city.world"
ardupilot_vehicle_path='~/ardupilot/Tools/autotest/sim_vehicle.py'

# Choose Simulator

In [4]:
simulator ='gazebo' # 'QGroundControl' #

## Gonfiguration

In [5]:
offsets = [ #east, north, up, heading
    (40, -10, 0, 0), 
    (40,  20, 0, 45)]
n_uavs = len(offsets)

wps = np.array([(40, -10, 5),
                (15, -10, 5),
                (0, -10, 5),
                (0,   4, 5),
                (40, 20, 5),
                (15, 20, 5),
                (0,  20, 5)])

homes=np.array([offset[:3] for offset in offsets])
target_wps=np.array([
                (0, 4,5),
                (0, 4,5)])

targets=np.array([(*target_wp[:2],0) for target_wp in target_wps])
markers = {'waypoints': {'pos':wps,'color':'yellow'},
           'homes': {'pos':homes,'color':'blue'},
           'targets':{'pos':targets,'color':'green'}}
# Interactive widget to rotate the plot

plot_3d_interactive(markers,title='Simulation Markers',expand=[0.2,0.2,0.4],ground=-0.05)


if simulator == 'QGroundControl':
    # lat,long,alt,heading
    home_position=(-35.3633245,149.1652241,0,90)
    spawns=find_spawns(home_position, offsets)

## Get Local positions

In [6]:
local_wps=global2local(wps, homes,pairwise=False)
local_target_wps=global2local(target_wps, homes,pairwise=True)

## Waypoint seletion algorithm

The path is shown in local coordinates

In [7]:
for i,(target_wp,wps) in enumerate(zip(local_target_wps,local_wps)):
        local_path=find_path(np.array((0,0,0)),target_wp, wps,eps=1)
        print(f'uav-{i} Best path: {local_path}') 


uav-0 Best path: [[  0   0   0]
 [  0   0   5]
 [-25   0   5]
 [-40   0   5]
 [-40  14   5]]
uav-1 Best path: [[  0   0   0]
 [  0   0   5]
 [-25   0   5]
 [-40   0   5]
 [-40 -16   5]]


Launch Simulator

In [8]:
if simulator == 'QGroundControl':
    sim_path = os.path.expanduser(QGC_path)
    sim_cmd =[sim_path]
elif simulator == 'gazebo':
    # Convert to Gazebo format (name, x, y, z, roll, pitch, yaw)
    drones = [(east, north, up, 0, 0, heading_to_yaw(heading)) for i, (east, north, up, heading) in enumerate(offsets)]
    world_path = os.path.expanduser(gazebo_word_path)
    updated_world_path = update_world(drones,markers,world_path)
    sim_cmd = ["gazebo", "--verbose", updated_world_path] 

simulator_process = subprocess.Popen(
                    sim_cmd,
                    stdout=subprocess.DEVNULL,  # Suppress standard output
                    stderr=subprocess.DEVNULL,  # Suppress error output
                    shell=False  # Ensure safety when passing arguments
                    )

Create uavs

In [9]:
uavs=[]
for i,home in enumerate(zip(homes)):
    uavs.append(VehicleLogic(sys_id=i+1,
                            home=home))

Plan 'basic plan' created — no connection yet 🧩
Action 'Pre-Arm Check' created — no connection yet 🧩
Step 'Check disarmed' created — no connection yet 🧩
Step 'Check EKF' created — no connection yet 🧩
Step 'Check GPS' created — no connection yet 🧩
Step 'Check system' created — no connection yet 🧩
Action 'Set Mode: GUIDED' created — no connection yet 🧩
Step 'Switch to GUIDED' created — no connection yet 🧩
Action 'Arm' created — no connection yet 🧩
Step 'arm' created — no connection yet 🧩
Action 'Take Off' created — no connection yet 🧩
Step 'take off' created — no connection yet 🧩
Action 'Land' created — no connection yet 🧩
Step 'land' created — no connection yet 🧩
vehicle 1 created
Plan 'basic plan' created — no connection yet 🧩
Action 'Pre-Arm Check' created — no connection yet 🧩
Step 'Check disarmed' created — no connection yet 🧩
Step 'Check EKF' created — no connection yet 🧩
Step 'Check GPS' created — no connection yet 🧩
Step 'Check system' created — no connection yet 🧩
Action 'Set Mo


Launch Vehicle

In [10]:
vehicle_path = os.path.expanduser(ardupilot_vehicle_path)
for i in range(n_uavs):
    vehicle_cmd = f"python3 {vehicle_path} -v ArduCopter -I{i} --sysid {i+1} --no-rebuild"
    if simulator == 'QGroundControl':
        spawn=','.join(map(str, spawns[i]))
        vehicle_cmd += f" --custom-location={spawn}"
    elif simulator == 'gazebo':
        vehicle_cmd += " -f gazebo-iris"
    subprocess.Popen(["gnome-terminal", "--", "bash", "-c", f"{vehicle_cmd}; exec bash"])

## Execute Plan

In [ ]:
from plan.core import State
from plan.actions.navegation import get_local_position

<Step 'Check disarmed' — State: NOT_STARTED>

In [ ]:
mission_incomplete = n_uavs*[True]
paths=[[] for _ in range(n_uavs)]

while any(mission_incomplete):
    for i,uav in enumerate(uavs):
        current_action=uav.current_action()
        if current_action.name=="Take Off" and current_action.state == State.DONE:
            # Search algorithm
            final_wp = local_target_wps[i]
            curr_wp  = get_local_position(uav.wps[0]

           
            # Search algorithm
            curr_wp= find_best_waypoint(curr_wp, final_wp, local_wps[i],eps=eps)
            
            # Modify wp_plan
            if np.array_equal(curr_wp, final_wp):
                path=np.stack([final_wp],axis=0)
            else:
                path=np.stack([curr_wp,final_wp],axis=0)

            uav.update_waypoints(path)

            paths[i].append(curr_wp)
        mission_incomplete[i] = uav.act_plan()
for path in paths:
    print(path)
        

vehicle 1: action check_prearm is done
vehicle 2: action check_prearm is done
vehicle 1: action check_pos_est is done
vehicle 2: action check_pos_est is done
vehicle 1: action mode_stabilize is done
vehicle 2: action mode_stabilize is done
vehicle 1: action mode_guided is done
vehicle 2: action mode_guided is done
vehicle 1: action arm is done
vehicle 2: action arm is done
--- uav 0 rached waypoint 1 --- 
--- uav 1 rached waypoint 1 --- 
--- uav 0 rached waypoint 2 --- 
--- uav 1 rached waypoint 2 --- 
--- uav 0 rached waypoint 3 --- 
--- uav 1 rached waypoint 3 --- 
vehicle 1: action land is done
vehicle 2: action land is done
[array([-25,   0,   5]), array([-40,   0,   5]), array([-40,  14,   5])]
[array([-25,   0,   5]), array([-40,   0,   5]), array([-40, -16,   5])]


## Kill all related process

In [ ]:
# for process in ["QGroundControl", "sim_vehicle.py", "arducopter", "gazebo", "mavproxy"]:
#     os.system(f"pkill -9 -f {process}")